# U.S. Stock Market II: Income vs Revenue

In [ ]:
import pandas as pd
import numpy as np
import requests, zipfile, io
import os
from pathlib import Path

from tiingo import TiingoClient                       
tiingo = TiingoClient({'api_key':'XXXX'})

# Plotting:
import matplotlib.pyplot as plt                        # Basic plot library.
plt.style.use('ggplot')                                # Make plots look nice.

In [ ]:
ef get_items_from_SEC_files(tags, filename=None):                # Function input: list of tags, optional filename.

    directory = 'data/sec/merged/'                                # Read data from here.
    filenames = [filename] if filename else os.listdir(directory) # Supplied filename or all files in "merged" directory.
    filenames = [f for f in filenames if not f.startswith(".")]   # Exclude hidden files from file list.

    results   = {t:pd.DataFrame() for t in tags}                  # Dictionary of tables (1 table for each tag)

    for filename in filenames:                                    # Loop over all files.
        print(filename)
        data = pd.read_csv(directory+filename, parse_dates=['filed','ddate'])  # Read the file.
        
        for t in tags:                                            # Loop over all tags.
            item  = data[data.tag==t]                             # Select all data for this tag.
            short = item.sort_values(['cik','filed','ddate','qtrs'], ascending=[True,True,True,False]) # Samllest qrts.
            long  = item.sort_values(['cik','filed','ddate','qtrs'], ascending=[True,True,True,True])  # Largest  qtrs.
            short = short.groupby(['cik','filed']).last()[['value','qtrs']]     # One value for each firm and filing.
            long  = long .groupby(['cik','filed']).last()[['value','qtrs']]     
            short_long = short.join(long, lsuffix='_shortest', rsuffix='_longest') # Put shortest and longest next to each other.
            results[t] = results[t].append( short_long )  
                        
    for t in tags:                                                # Now sort all tables by filing date.
        if not results[t].empty: results[t] = results[t].sort_index(level='filed')            

    return results



def combine_items(tags, items):
    result = items[tags[0]]
    for tag in tags[1:]:  result = result.combine_first( items[tag] )
    return result



def calculate_quarterly_annual_values(item):                        # item: table with shortest and longest values and quarters.
    result           = pd.DataFrame()                               # Results go here.
    all_firms        = item.index.get_level_values('cik').unique()  # All CIKs.
    all_filing_dates = pd.read_csv('data/sec/dates/filing_dates.csv', index_col='cik', parse_dates=['filed'])
    
    for cik in all_firms:                                           # Loop over all firms.  
        filing_dates = pd.Series(all_filing_dates.filed[cik])       # All filing dates for this firm.

        # Quarterly values:
        valuesQ = item.loc[cik].value_shortest.reindex(filing_dates) # Values with shortest reported quarters.
        qtrsQ   = item.loc[cik].qtrs_shortest.astype(int)           # Number of quarters for each value.
        for date,q in qtrsQ[qtrsQ>1].iteritems():                   # Loop over all dates with > 1 quarters. 
            previous_values = valuesQ[:date][-q:-1]                 # Example: for q=3 we need to subtract 2 previous quarters.            
            if len(previous_values) == q-1:                         # If all previous values available.
                valuesQ[date] -= previous_values.sum(skipna=False)  # Subtract previous values to get quarterly value.
            else:
                valuesQ[date]  = np.nan                  

        # Annual values:
        valuesA = item.loc[cik].value_longest.reindex(filing_dates) # Values with longest reported quarters.
        qtrsA   = item.loc[cik].qtrs_longest.astype(int)            # Number of quarters for each value.
        for date,q in qtrsA[qtrsA<4].iteritems():                   # Loop over all dates with < 4 quarters. 
            previous_values = valuesQ[:date][-4:-q]                 # Example: for q=2 we need to add quarters -3 and -4.
            if len(previous_values) == 4-q:                         # If all previous data available.     
                valuesA[date] += previous_values.sum(skipna=False)  # Add previous values to get annual values.
            else:
                valuesA[date]  = np.nan        
        
        result = result.append( pd.DataFrame({'cik':cik, 'filed':filing_dates, 'valueQ':valuesQ.values, 'valueA':valuesA.values}) )

    return result.set_index(['cik','filed'])                        # Return a table with columns 'valueQ' and 'valueA'.



def ffill_values(item, dates):                                          
    data = item.unstack('cik')
    data = data.reindex(dates.union(data.index)).sort_index()           # Add specified dates to index.
    filing_dates = pd.read_csv('data/sec/dates/filing_dates.csv', index_col='cik', parse_dates=['filed']).filed
    last_filing_date_all_firms = filing_dates.max()                     # Most recent date where at least 1 firm filed.
     
    for cik in data.columns:                                            # Loop over all firms.
        last_filing_date      = pd.Series(filing_dates[cik]).iloc[-1]   # Last date where this firm filed
        days_since_last_filed = (last_filing_date_all_firms - last_filing_date).days
        last_date_this_firm   = dates[-1] if days_since_last_filed < 120 else last_filing_date
        data.loc[:last_date_this_firm, cik].ffill(inplace=True)         # Forward fill all the values.

    return data.loc[dates]                                              # Return only specified dates.   

In [ ]:
tags_sales           = ['RevenueFromContractWithCustomerExcludingAssessedTax','SalesRevenueNet','Revenues',
                        'SalesAndOtherOperatingRevenueIncludingSalesBasedTaxes','TotalRevenuesAndOtherIncome']

tags_earnings        = ['NetIncomeLossAvailableToCommonStockholdersBasic','NetIncomeLoss','ProfitLoss']

tags_operatingIncome = ['OperatingIncomeLoss']

In [ ]:
items = get_items_from_SEC_files( tags_sales + tags_earnings + tags_operatingIncome )

Fix data errors:

In [ ]:
# RUN THIS CELL ONLY ONCE!

# MKSI cik: 1049502
tag = 'RevenueFromContractWithCustomerExcludingAssessedTax'
t = items[tag].reset_index()
t.loc[(t.cik==1049502) & (t.filed=='2018-05-08'), 'value_longest']  /= 1000
t.loc[(t.cik==1049502) & (t.filed=='2018-05-08'), 'value_shortest'] /= 1000
items[tag] = t.set_index(['cik','filed'])

# CRAWA cik: 1049502: Income, 6 Months Ended 2013-3-31 = $263,235
# https://www.sec.gov/cgi-bin/viewer?action=view&cik=47307&accession_number=0000047307-13-000009&xbrl_type=v#
tag = 'NetIncomeLossAvailableToCommonStockholdersBasic'
t = items[tag].reset_index()
t.loc[(t.cik==47307) & (t.filed=='2013-5-15'), 'value_longest'] /= 10**6
items[tag] = t.set_index(['cik','filed'])

# Bonanza Creek Energy, Inc; CIK=1509589 -> OperatingIncomeLoss 2013-03-15 = 3.666547e+7 (/1000 )
# WMT CIK=104169, 2017-08-31 values missing, source: https://www.sec.gov/cgi-bin/viewer?action=view&cik=104169&accession_number=0000104169-17-000057&xbrl_type=v#
tag = 'OperatingIncomeLoss'
t = items[tag].reset_index()
t.loc[(t.cik==1509589) & (t.filed=='2013-03-15'), 'value_shortest'] /= 1000
t.loc[(t.cik==104169)  & (t.filed=='2017-08-31'), ['value_shortest','qtrs_shortest','value_longest','qtrs_longest']] = [5969*10**6, 1, 11440, 2]
items[tag] = t.set_index(['cik','filed'])

Combine items:

In [ ]:
items['Sales']    = combine_items(tags_sales,    items)
items['Earnings'] = combine_items(tags_earnings, items)

Calculate quarterly and annual values:

In [ ]:
sales            = calculate_quarterly_annual_values(items['Sales'])
earnings         = calculate_quarterly_annual_values(items['Earnings'])
operatingIncome  = calculate_quarterly_annual_values(items['OperatingIncomeLoss'])

sales[:5]

Save the results:

In [ ]:
sales          .to_csv('data/sec/items/Sales.csv')
earnings       .to_csv('data/sec/items/Earnings.csv')
operatingIncome.to_csv('data/sec/items/OperatingIncome.csv')

In [ ]:
trading_days = tiingo.get_dataframe('SPY','2009-04-15').index.tz_convert(None)

salesQ = ffill_values(sales.valueQ, trading_days)
salesA = ffill_values(sales.valueA, trading_days)

earningsQ = ffill_values( earnings.valueQ, trading_days )  
earningsA = ffill_values( earnings.valueA, trading_days )

operatingIncomeQ = ffill_values( operatingIncome.valueQ, trading_days )  
operatingIncomeA = ffill_values( operatingIncome.valueA, trading_days )

salesA[:5]

Which 10 firms have the most recent highest annual sales?

In [ ]:
symbols = pd.read_csv('data/ticker_symbols/symbols.csv',index_col=0)
symbols[:2]

Which 10 firms have the most recent highest annual earnings?

Aggregate quarterly earnings:

Earnings relative to sales for a specific firm:

In [ ]:
cik = symbols[symbols.ticker==''].index[0]



Earnings relative to operating income for the entire market:

Earnings relative to operating income:

How to select values that multiple tables have in common:

In [ ]:
t1 = pd.DataFrame({'A':[1,2,3],'B':[4,5,np.nan],'C':[7,8,9]}, index=['a','b','c'])
t1

In [ ]:
t2 = pd.DataFrame({'B':[1,2,3],'C':[np.nan,5,6],'D':[7,8,9]}, index=['b','c','d'])
t2

In [ ]:
mask = 

total_operatingIncome = operatingIncomeQ[mask].sum('columns')
total_earnings        = earningsQ       [mask].sum('columns')

(total_earnings / total_operatingIncome)['2012':].plot()

Same for sales:

In [ ]:
mask = 

total_sales     = salesQ   [mask].sum('columns')
total_earnings  = earningsQ[mask].sum('columns')

(total_earnings / total_sales)['2012':].plot()